#  １章 MNIST

In [ ]:
%matplotlib inline
from IPython.display import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns

import keras
from keras import layers
from keras import models

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

from sklearn.model_selection import train_test_split
x_train1, x_valid, y_train1, y_valid = train_test_split(x_train, y_train, test_size=0.175)

In [ ]:
# MNISTデータの表示
fig = plt.figure(figsize=(9, 9))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)
for i in range(81):
    ax = fig.add_subplot(9, 9, i + 1, xticks=[], yticks=[])
    ax.imshow(x_train[i].reshape((28, 28)), cmap='gray')

### いろいろなモデルでやつてみる

In [ ]:
# モデル1
model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # (28, 28, 1) ⇒ (26, 26, 32)
model1.add(layers.MaxPooling2D((2, 2)))                                           # (26, 26, 32) ⇒ (13, 13, 32)
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.Flatten())
model1.add(layers.Dense(64, activation='relu'))
model1.add(layers.Dense(10, activation='softmax'))

model1.summary()

In [ ]:
# モデル2  過学習対策用のDropout

model2 = models.Sequential()
model2.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Dropout(0.25))
model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(10, activation='softmax'))

model2.summary()


In [ ]:
#  モデル3  さらに多段

model3 = models.Sequential()
model3.add(layers.Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=(28, 28, 1)))
model3.add(layers.Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
model3.add(layers.MaxPool2D((2, 2)))
model3.add(layers.Dropout(0.20))
model3.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model3.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model3.add(layers.MaxPool2D(pool_size=(2, 2)))
model3.add(layers.Dropout(0.25))
model3.add(layers.Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model3.add(layers.Dropout(0.25))
model3.add(layers.Flatten())
model3.add(layers.Dense(128, activation='relu'))
model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.25))
model3.add(layers.Dense(10, activation='softmax'))

model3.summary()

### 前処理

In [ ]:
# 28 x 28の画像がgrayscaleで1chなので、28, 28, 1にreshapeする
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_valid = x_valid.reshape(x_valid.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# 0-255の整数値を0〜1の小数に変換する
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_valid /= 255
x_test /= 255

# one-hot vector形式に変換する
y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)
y_test = keras.utils.to_categorical(y_test, 10)

### 学習 描画など　定義

In [ ]:
from keras.optimizers import RMSprop

def gaku(model, epo, bsize):
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                        batch_size=bsize,
                        epochs=epo,
                        verbose=1,
                        validation_data=(x_valid, y_valid))
    
    return history

def viewplot(model, history, name):
    # 精度のplot
    plt.plot(history.history['acc'], marker='.', label='acc')
    plt.plot(history.history['val_acc'], marker='.', label='val_acc')
    plt.title('%s model accuracy' % name)
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.savefig('MNIST_acc_%s.png' % name)
    plt.show()
    
    # 損失のplot
    plt.plot(history.history['loss'], marker='.', label='loss')
    plt.plot(history.history['val_loss'], marker='.', label='val_loss')
    plt.title('%s model loss' % name)
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.savefig('MNIST_loss_%s.png' % name)
    plt.show()

In [ ]:
history = gaku(model1, 5, 128) #  MacBook で　１分/epoch、全部で5分ほどかかる

In [ ]:
viewplot(model1, history, 'model1')

In [ ]:
score = model1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# GPU付きPCでの実行結果例
Image("model1_gpu.png")

### 混同行列(Confusion Matrix) とは2 値分類問題で出力されたクラス分類の結果をまとめたマトリックスのことで、2 値分類機械学習モデルの性能を測る指標として使われる

### ただし混同行列はone-hot化されたベクトルデータではNGなので、元に戻す必要がある

In [ ]:
# one-hot 化されたベクトルを、one-hot化される前のリストに戻すには　こうする！
y_test_org = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

def show_matrix(y_true, y_pred):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    plt.figure(figsize = (12,7))
    sns.heatmap(df_cmx, annot=True, cmap='Blues' ,fmt='g',  square = True)
    plt.show()

In [ ]:
predict_classes = model1.predict_classes(x_test)
true_classes = y_test_org

show_matrix(true_classes, predict_classes)

### 学習 モデル２

In [ ]:
# GPUなしだと大変時間がかかる　　3回回せば良いか？　本当なら２０回は回す？  MacBookで３分/epoch

history = gaku(model2, 3, 128)

In [ ]:
viewplot(model2, history, 'model2')

In [ ]:
score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# GPU付きPCでの実行結果例
Image('model2_gpu.png')

In [ ]:
# one-hot 化されたベクトルを、one-hot化される前のリストに戻す
y_test_org = np.argmax(y_test, axis=1)

predict_classes = model2.predict_classes(x_test)
true_classes = y_test_org

show_matrix(true_classes, predict_classes)

In [ ]:
# GPUなしだと大変時間がかかる　3回くらいでどうか？ 本当は２０回くらいがいいか。　Macbookで　４分／epoch

history = gaku(model3, 3, 128)

In [ ]:
viewplot(model3, history, 'model3')

In [ ]:
score = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# GPU付きPCでの実行結果例
Image('model3_gpu.png')

In [ ]:
# one-hot 化されたベクトルを、one-hot化される前のリストに戻す
y_test_org = np.argmax(y_test, axis=1)

predict_classes = model3.predict_classes(x_test)
true_classes = y_test_org

show_matrix(true_classes, predict_classes)

# ２章 Fashion MNIST  

In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix

In [ ]:
# x_train・・・60,000枚の訓練セット
# x_test・・・10,000枚のテストセット
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train.shape

In [ ]:
x_test.shape

In [ ]:
plt.figure(figsize=(12,15))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.title("Label: " + str(i))
    plt.imshow(x_train[i].reshape(28,28))

In [ ]:
y_train[0:25]

In [ ]:
x_train = x_train.reshape((60000, 28 * 28))
x_train = x_train.astype('float32') / 255
 
x_test = x_test.reshape((10000, 28 * 28))
x_test = x_test.astype('float32') / 255

def graph_view(history):
    # 精度のプロット
    plt.plot(history.history['acc'],"-",label="accuracy")
    plt.plot(history.history['val_acc'],"-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()
 
    # 損失のプロット
    plt.plot(history.history['loss'],"-",label="loss",)
    plt.plot(history.history['val_loss'],"-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.optimizers import RMSprop
 
model = keras.models.Sequential()
model.add(Dense(units=512,input_dim=28*28))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(units=10))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=40,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [ ]:
graph_view(history)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
 
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
predict_classes = model.predict_classes(x_test)
true_classes = y_test

show_matrix(true_classes, predict_classes)

### 対角線上にない数字が、誤認識している箇所

# ３章  犬と猫のKaggleデータによるCNN

データの場所：
https://www.kaggle.com/c/dogs-vs-cats/data

In [ ]:
import os, shutil

In [ ]:
original_dataset_dir = '/ご自身のインストールディレクトリパス/dogs-vs-cats/train'

base_dir = '/ご自身のインストールディレクトリパス/dogs-vs-cats/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

### 以下は最初に１回だけ行う

In [ ]:
# ディレクトリ位置はマシンごとに格納した場所に合わせる
os.mkdir(base_dir)

# Directories for our training,validation and test splits
os.mkdir(train_dir)
os.mkdir(test_dir)

# Directory with our training cat pictures
os.mkdir(train_cats_dir)

# Directory with our training dog pictures
os.mkdir(train_dogs_dir)

# Directory with our validation cat pictures
os.mkdir(validation_cats_dir)

# Directory with our validation dog pictures
os.mkdir(validation_dogs_dir)

# Directory with our validation cat pictures
os.mkdir(test_cats_dir)

# Directory with our validation dog pictures
os.mkdir(test_dogs_dir)

In [ ]:
# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

### モデル

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

from keras.preprocessing.image import ImageDataGenerator

# 全画像は 1./255　にスケーリングする
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # 訓練データのディレクトリ
        target_size=(150, 150), # 画像サイズは150x150にする
        batch_size=20,
        class_mode='binary') # binary 交差エントロピーを使用

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
# GPUなしだと時間はかかる　　２分/エポック　MacBook

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# GPUマシンでの実行結果例
Image('catsanddogs.png')

### 過学習(overfitting)の兆候がある： validation lossが上昇している。これはデータ数不足なども原因。
### そこで元の画僧をランダムに回転させるなどの変換を施して、学習させるデータを水増しする

In [ ]:
datagen = ImageDataGenerator(
      rotation_range=40,  # 画像をランダムに0～180度回転させる
      width_shift_range=0.2,  # ランダムに水平移動
      height_shift_range=0.2,  # ランダムに垂直移動
      shear_range=0.2,  # ランダムな等積変形
      zoom_range=0.2,  # 図形の内側をランダムにズーム
      horizontal_flip=True,  # 画像の半分を水平方向にランダムに反転
      fill_mode='nearest')

In [ ]:
from keras.preprocessing import image

fnames = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]

# 水増し画像選択
img_path = fnames[3]

# 画像を読んでサイズ変更
img = image.load_img(img_path, target_size=(150, 150))

# (150, 150, 3)のnumpy配列に変換
x = image.img_to_array(img)

# Reshape (1, 150, 150, 3)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

In [ ]:
# Dropout層も追加する
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

# GPU付きPCでやっても相当時間がかかる  22秒/エポック　　３分　macbook
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=10, # 本当は100エポックくらいだが
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
Image('catsanddogs2.png')

# ４章  学習済みモデルによる特徴抽出

### VGG16モデル

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

conv_base.summary()

### データ拡張は行わない特徴抽出方法。

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=10,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
Image('catsanddogs3.png')

### データ拡張を行う特徴抽出　（注：GPUマシンでないと遅い）

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
# 畳み込みベースは凍結する
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=１0,  # 本来なら30回以上
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
Image('catsanddogs4.png')

# ５章  ファインチューニング

### 凍結した畳み込みベースの出力側のいくつかを解凍し、新しくモデルに追加された部分（全結合）と解凍した層の両方で訓練する （注： GPUマシンでないと遅い）

In [ ]:
conv_base.summary()

### 最後の block5_conv1 から block5_conv3 までの３つを使う

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
model.save('cats_and_dogs_small_4.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
Image('finetuning.png')

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)